# Dropout 正则化技术

## 核心思想

Dropout（Srivastava et al., 2014）是一种简单有效的正则化技术。在训练过程中，随机"丢弃"（置零）一部分神经元的输出，迫使网络学习更鲁棒的特征。

## 工作原理

```
训练阶段：
- 每个神经元以概率 p 被临时"关闭"
- 保留的神经元输出需要除以 (1-p) 进行缩放（inverted dropout）

推理阶段：
- 所有神经元都参与计算
- 无需额外缩放（因为训练时已处理）
```

## 为什么有效

| 机制 | 解释 |
|------|------|
| 集成效果 | 相当于训练多个子网络的集成 |
| 减少共适应 | 神经元不能过度依赖其他特定神经元 |
| 特征冗余 | 迫使网络学习更分散、更鲁棒的特征表示 |

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

# 设置随机种子
tf.random.set_seed(42)
np.random.seed(42)

print(f"TensorFlow 版本: {tf.__version__}")

## 1. 基本用法

In [ ]:
# 构建带 Dropout 的神经网络
model = keras.models.Sequential([
    # 输入层展平
    keras.layers.Flatten(input_shape=(28, 28)),
    
    # 输入层 Dropout（通常使用较小的 rate，如 0.2）
    keras.layers.Dropout(rate=0.2),
    
    # 第一个隐藏层
    keras.layers.Dense(256, activation='relu', kernel_initializer='he_normal'),
    # 隐藏层 Dropout（通常使用 0.2-0.5）
    keras.layers.Dropout(rate=0.3),
    
    # 第二个隐藏层
    keras.layers.Dense(128, activation='relu', kernel_initializer='he_normal'),
    keras.layers.Dropout(rate=0.3),
    
    # 第三个隐藏层
    keras.layers.Dense(64, activation='relu', kernel_initializer='he_normal'),
    keras.layers.Dropout(rate=0.3),
    
    # 输出层（无 Dropout）
    keras.layers.Dense(10, activation='softmax')
])

model.summary()

## 2. 可视化 Dropout 效果

In [ ]:
def visualize_dropout(rate=0.5, input_size=100):
    """
    可视化 Dropout 对神经元的影响
    
    Parameters:
    -----------
    rate : float
        Dropout 比率（0到1之间）
    input_size : int
        输入维度大小
    """
    dropout_layer = keras.layers.Dropout(rate=rate)
    
    # 创建输入数据
    x = tf.ones((1, input_size))
    
    # 训练模式下的输出
    output_train = dropout_layer(x, training=True)
    
    # 推理模式下的输出
    output_inference = dropout_layer(x, training=False)
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # 原始输入
    axes[0].bar(range(input_size), x.numpy().flatten(), color='blue', alpha=0.7)
    axes[0].set_title('原始输入')
    axes[0].set_xlabel('神经元索引')
    axes[0].set_ylabel('激活值')
    axes[0].set_ylim(0, 2)
    
    # 训练模式
    colors = ['red' if v == 0 else 'green' for v in output_train.numpy().flatten()]
    axes[1].bar(range(input_size), output_train.numpy().flatten(), color=colors, alpha=0.7)
    axes[1].set_title(f'训练模式 (rate={rate})')
    axes[1].set_xlabel('神经元索引')
    axes[1].set_ylabel('激活值')
    axes[1].set_ylim(0, 2)
    
    dropped = np.sum(output_train.numpy() == 0)
    axes[1].text(0.5, 0.95, f'丢弃: {dropped}/{input_size} ({dropped/input_size*100:.1f}%)',
                transform=axes[1].transAxes, ha='center', fontsize=10)
    
    # 推理模式
    axes[2].bar(range(input_size), output_inference.numpy().flatten(), color='blue', alpha=0.7)
    axes[2].set_title('推理模式 (无 Dropout)')
    axes[2].set_xlabel('神经元索引')
    axes[2].set_ylabel('激活值')
    axes[2].set_ylim(0, 2)
    
    plt.tight_layout()
    plt.show()

# 可视化不同 Dropout 率的效果
for rate in [0.2, 0.5]:
    print(f"\nDropout rate = {rate}")
    visualize_dropout(rate=rate, input_size=50)

## 3. 训练与验证对比

In [ ]:
# 加载数据集
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

# 数据预处理
X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.0

print(f"训练集: {X_train.shape}")
print(f"验证集: {X_valid.shape}")
print(f"测试集: {X_test.shape}")

In [ ]:
def create_model(dropout_rate=0.0):
    """
    创建带可配置 Dropout 率的模型
    
    Parameters:
    -----------
    dropout_rate : float
        Dropout 比率，0 表示不使用 Dropout
    
    Returns:
    --------
    keras.Model
        编译好的模型
    """
    model = keras.models.Sequential([
        keras.layers.Flatten(input_shape=(28, 28)),
    ])
    
    for units in [256, 128, 64]:
        model.add(keras.layers.Dense(units, activation='relu', kernel_initializer='he_normal'))
        if dropout_rate > 0:
            model.add(keras.layers.Dropout(rate=dropout_rate))
    
    model.add(keras.layers.Dense(10, activation='softmax'))
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# 创建对比模型
model_no_dropout = create_model(dropout_rate=0.0)
model_with_dropout = create_model(dropout_rate=0.3)

print(f"无 Dropout 模型参数量: {model_no_dropout.count_params():,}")
print(f"有 Dropout 模型参数量: {model_with_dropout.count_params():,}")

In [ ]:
# 训练对比
EPOCHS = 30
BATCH_SIZE = 64

print("训练无 Dropout 模型...")
history_no_dropout = model_no_dropout.fit(
    X_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_valid, y_valid),
    verbose=0
)
print("完成")

print("训练有 Dropout 模型...")
history_with_dropout = model_with_dropout.fit(
    X_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_valid, y_valid),
    verbose=0
)
print("完成")

In [ ]:
# 绘制训练曲线对比
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 准确率
axes[0].plot(history_no_dropout.history['accuracy'], 'b-', label='无 Dropout (训练)', alpha=0.8)
axes[0].plot(history_no_dropout.history['val_accuracy'], 'b--', label='无 Dropout (验证)', alpha=0.8)
axes[0].plot(history_with_dropout.history['accuracy'], 'r-', label='有 Dropout (训练)', alpha=0.8)
axes[0].plot(history_with_dropout.history['val_accuracy'], 'r--', label='有 Dropout (验证)', alpha=0.8)
axes[0].set_title('准确率对比')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 损失
axes[1].plot(history_no_dropout.history['loss'], 'b-', label='无 Dropout (训练)', alpha=0.8)
axes[1].plot(history_no_dropout.history['val_loss'], 'b--', label='无 Dropout (验证)', alpha=0.8)
axes[1].plot(history_with_dropout.history['loss'], 'r-', label='有 Dropout (训练)', alpha=0.8)
axes[1].plot(history_with_dropout.history['val_loss'], 'r--', label='有 Dropout (验证)', alpha=0.8)
axes[1].set_title('损失对比')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('dropout_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

# 计算过拟合程度
gap_no_dropout = history_no_dropout.history['accuracy'][-1] - history_no_dropout.history['val_accuracy'][-1]
gap_with_dropout = history_with_dropout.history['accuracy'][-1] - history_with_dropout.history['val_accuracy'][-1]

print(f"\n过拟合程度（训练-验证准确率差）:")
print(f"无 Dropout: {gap_no_dropout:.4f}")
print(f"有 Dropout: {gap_with_dropout:.4f}")

In [ ]:
# 测试集评估
print("测试集评估:")
test_loss_no, test_acc_no = model_no_dropout.evaluate(X_test, y_test, verbose=0)
test_loss_with, test_acc_with = model_with_dropout.evaluate(X_test, y_test, verbose=0)

print(f"无 Dropout: 准确率 = {test_acc_no:.4f}, 损失 = {test_loss_no:.4f}")
print(f"有 Dropout: 准确率 = {test_acc_with:.4f}, 损失 = {test_loss_with:.4f}")

## 4. Dropout 变体

### 4.1 空间 Dropout (Spatial Dropout)

用于卷积神经网络，丢弃整个特征图而非单个像素。

In [ ]:
# Spatial Dropout 示例（用于 CNN）
cnn_model = keras.models.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    # SpatialDropout2D 丢弃整个特征图
    keras.layers.SpatialDropout2D(0.25),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.SpatialDropout2D(0.25),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation='softmax')
])

print("CNN with Spatial Dropout:")
cnn_model.summary()

### 4.2 Alpha Dropout（用于 SELU）

专为自归一化网络设计，保持输入的均值和方差。

In [ ]:
# Alpha Dropout 示例（用于 SELU 激活的自归一化网络）
selu_model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(256, activation='selu', kernel_initializer='lecun_normal'),
    keras.layers.AlphaDropout(0.1),  # SELU 专用
    keras.layers.Dense(128, activation='selu', kernel_initializer='lecun_normal'),
    keras.layers.AlphaDropout(0.1),
    keras.layers.Dense(10, activation='softmax')
])

print("SELU Network with Alpha Dropout:")
selu_model.summary()

## 5. 使用建议

### Dropout 率选择

| 层类型 | 推荐 Dropout 率 |
|--------|----------------|
| 输入层 | 0.1 - 0.2 |
| 隐藏层 | 0.2 - 0.5 |
| 输出层 | 不使用 Dropout |
| CNN 特征提取层 | 0.25 (Spatial Dropout) |
| CNN 全连接层 | 0.5 |

### 注意事项

1. **与 BN 配合**: 通常将 Dropout 放在 BN 之后，或不同时使用
2. **大数据集**: 数据量大时，可能不需要 Dropout
3. **测试时**: 确保 `training=False`，或使用 `model.predict()`
4. **率过高**: Dropout 率过高会导致欠拟合

In [ ]:
# 验证代码正确性
print("Dropout 模块测试完成")
print("\n关键要点:")
print("1. Dropout 随机丢弃神经元，防止过拟合")
print("2. 训练时应用 Dropout，推理时关闭")
print("3. 输入层用小 rate (0.1-0.2)，隐藏层用中等 rate (0.2-0.5)")
print("4. SELU 激活函数配合 AlphaDropout 使用")